In [ ]:
import csv
import numpy as np
samples = {}
ALPHABET = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+ =<>()[]{}"
FEATURE_LEN =1014
cdict = {}
for i,c in enumerate(ALPHABET):
    cdict[c] = i + 2

In [ ]:
samples = {}
with open('dbpedia_data/train.csv') as f:
    reader = csv.DictReader(f,fieldnames=['class'],restkey='fields')
    for row in reader:
        label = row['class']
        if label not in samples:
            samples[label] = []
        sample = np.ones(FEATURE_LEN)
        count = 0
        for field in row['fields']:
            for char in field.lower():
                if char in cdict:
                    sample[count] = cdict[char]
                    count += 1
                if count >= FEATURE_LEN-1:
                    break
        samples[label].append(sample)
    samples_per_class = None
    classes = samples.keys()
    class_samples = []
    for c in classes:
        if samples_per_class is None:
            samples_per_class = len(samples[c])
        else:
            assert samples_per_class == len(samples[c])
        class_samples.append(samples[c])

In [ ]:
def build_onehot(input_):
    target = np.zeros(len(classes))
    target[input_] = 1
    return target
y= []
for i in range(len(classes)):
    for j in range(samples_per_class):
        target =build_onehot(i)
        y.append(target)

In [ ]:
x = np.reshape(class_samples,(-1,FEATURE_LEN))

In [ ]:
y = np.reshape(y,(-1,len(classes)))

In [ ]:
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]
# Split train/test set
n_dev_samples = 10000
# TODO: Create a fuckin' correct cross validation procedure
x_train, x_dev = x_shuffled[:-n_dev_samples], x_shuffled[-n_dev_samples:]
y_train, y_dev = y_shuffled[:-n_dev_samples], y_shuffled[-n_dev_samples:]
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

In [ ]:
import tensorflow as tf
from inception.slim import ops
import tflearn
class CharCNN(object):
    """
    A CNN for text classification.
    based on the Very Deep Convolutional Networks for Natural Language Processing.
    """
    def __init__(self, num_classes=14, filter_size=3,
                 l2_reg_lambda=0.001, sequence_max_length=1014, num_quantized_chars=71,embedding_size=16):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None,sequence_max_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        self.training =  tf.placeholder(tf.int32, name="trainable")
        if self.training==1:
            TRAIN = True
        else:
            TRAIN = False
        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)
        # ================ embedding character ================
        with tf.device('/cpu:0'),tf.name_scope("embedding"):
            W0 = tf.Variable(tf.random_uniform([num_quantized_chars, embedding_size], -1.0, 1.0),name="W")
            self.embedded_characters = tf.nn.embedding_lookup(W0,self.input_x)
            self.embedded_characters_expanded = tf.expand_dims(self.embedded_characters,-1,name="embedding_input")
        # ================ Layer 1 ================
        with tf.name_scope("conv-maxpool-1"):
            num_filters_layer_1 = 64
            filter_shape = [3, embedding_size, 1, num_filters_layer_1]
            filter1 = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.05), name="filter1")
            self.conv1 = tf.nn.conv2d(self.embedded_characters_expanded, filter1, strides=[1, 1,embedding_size, 1], padding="SAME",
                                     name="conv1")
            self.batch_normal0=tflearn.layers.normalization.batch_normalization(self.conv1,trainable= TRAIN,
                                                                                name="BatchNormalization0")
           
        # ================ Layer 2 ================
        with tf.name_scope("conv-maxpool-2_3"):
            filter_shape1 = [3, 1, 64, 64]
            filter_shape2 = [3, 1, 64, 64]
            filter_1 = tf.Variable(tf.truncated_normal(filter_shape1, stddev=0.05), name="W1")
            filter_2 = tf.Variable(tf.truncated_normal(filter_shape2, stddev=0.05), name="W2")
            conv1 = tf.nn.conv2d(self.batch_normal0, filter_1, strides=[1, 1, filter_shape1[1], 1], padding="SAME")
            batch_normal1 =tflearn.layers.normalization.batch_normalization(conv1,trainable= TRAIN,name="BatchNormalization1")
            h1 = tf.nn.relu(batch_normal1)
            self.conv2 = tf.nn.conv2d(h1, filter_2, strides=[1, 1, filter_shape2[1], 1], padding="SAME")
            batch_normal2 =tflearn.layers.normalization.batch_normalization(self.conv2,trainable=TRAIN,name="BatchNormalization2")
            h2 = tf.nn.relu(batch_normal2)
            self.pooled1 = tf.nn.max_pool(h2,ksize=[1, 3, 1, 1],strides=[1, 2, 1, 1],padding='SAME',name="pool1")
            
        # ================ Layer 3 ================
        with tf.name_scope("conv-maxpool-6_7_8_9"):
            filter_shape3 = [3, 1, 64,128]
            filter_shape4 = [3, 1, 128, 128]
            filter_3 = tf.Variable(tf.truncated_normal(filter_shape3, stddev=0.05), name="W3")
            filter_4 = tf.Variable(tf.truncated_normal(filter_shape4, stddev=0.05), name="W4")
            conv4 = tf.nn.conv2d(self.pooled1, filter_3, strides=[1, 1, filter_shape3[1], 1], padding="SAME")
            batch_normal3 =tflearn.layers.normalization.batch_normalization(conv4,trainable= TRAIN,name="BatchNormalization3")
            h3= tf.nn.relu(batch_normal3)
            conv5 = tf.nn.conv2d(h3, filter_4, strides=[1, 1, filter_shape4[1], 1], padding="SAME")
            batch_normal4 =tflearn.layers.normalization.batch_normalization(conv5,trainable= TRAIN,name="BatchNormalization4")
            h4 = tf.nn.relu(batch_normal4)
            self.pooled2 = tf.nn.max_pool(h4,ksize=[1, 3, 1, 1],strides=[1, 2, 1, 1],padding='SAME',name="pool1")
        # ================ Layer 4 ================
        with tf.name_scope("conv-maxpool-10_11_12_13"):
            filter_shape5 = [3, 1, 128,256]
            filter_shape6 = [3, 1, 256, 256]
            filter_5 = tf.Variable(tf.truncated_normal(filter_shape5, stddev=0.05), name="W3")
            filter_6 = tf.Variable(tf.truncated_normal(filter_shape6, stddev=0.05), name="W4")
            conv6 = tf.nn.conv2d(self.pooled2, filter_5, strides=[1, 1, filter_shape5[1], 1], padding="SAME")
            batch_normal5 =tflearn.layers.normalization.batch_normalization(conv6,trainable= TRAIN,name="BatchNormalization5")
            h5= tf.nn.relu(batch_normal5)
            conv7 = tf.nn.conv2d(h5, filter_6, strides=[1, 1, filter_shape6[1], 1], padding="SAME")
            batch_normal6 =tflearn.layers.normalization.batch_normalization(conv7,trainable= TRAIN,name="BatchNormalization6")
            h6= tf.nn.relu(batch_normal6)
            self.pooled3 = tf.nn.max_pool(h6,ksize=[1, 3, 1, 1],strides=[1, 2, 1, 1],padding='SAME',name="pool1")
        # ================ Layer 5 ================
        with tf.name_scope("conv-maxpool-14_15_16_17"):
            filter_shape7 = [3, 1, 256,512]
            filter_shape8 = [3, 1, 512, 512]
            filter_7 = tf.Variable(tf.truncated_normal(filter_shape7, stddev=0.05), name="W3")
            filter_8 = tf.Variable(tf.truncated_normal(filter_shape8, stddev=0.05), name="W4")
            conv8 = tf.nn.conv2d(self.pooled3, filter_7, strides=[1, 1, filter_shape7[1], 1], padding="SAME")
            batch_normal7 =tflearn.layers.normalization.batch_normalization(conv8,trainable= TRAIN,name="BatchNormalization7")
            h7= tf.nn.relu(batch_normal7)
            conv9 = tf.nn.conv2d(h7, filter_8, strides=[1, 1, filter_shape8[1], 1], padding="SAME")
            batch_normal8 =tflearn.layers.normalization.batch_normalization(conv9,trainable= TRAIN,name="BatchNormalization8")
            h8= tf.nn.relu(batch_normal8)
            self.pooled4 = tf.nn.max_pool(h8,ksize=[1, 13, 1, 1],strides=[1, 13, 1, 1],padding='VALID',name="pool3")
            self.after_k_maxpool = tf.reshape(self.pooled4,(-1,512*9))
        # ================ fc-3 ================
        # I just use 2 layers
        with tf.name_scope("fc-1"):
            W1 = tf.Variable(tf.truncated_normal([4608, 2048], stddev=0.05), name="W1")
            b1 = tf.Variable(tf.constant(0.1, shape=[2048]), name="b1")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.fc_1_output = tf.nn.relu(tf.nn.xw_plus_b(self.after_k_maxpool, W1, b1), name="fc-1-out")
        '''
        with tf.name_scope("fc-2"):
            W2 = tf.Variable(tf.truncated_normal([2048, 2048], stddev=0.05), name="W2")
            b2 = tf.Variable(tf.constant(0.1, shape=[2048]), name="b2")
            # l2_loss += tf.nn.l2_loss(W)
            # l2_loss += tf.nn.l2_loss(b)
            self.fc_2_output = tf.nn.relu(tf.nn.xw_plus_b(self.fc_1_output, W2, b2), name="fc-2-out")
        '''
        with tf.name_scope("fc-3"):
            W3 = tf.Variable(tf.truncated_normal([2048, num_classes], stddev=0.05), name="W3")
            b3= tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b3")
            l2_loss += tf.nn.l2_loss(W3)
            l2_loss += tf.nn.l2_loss(b3)
            self.scores = tf.nn.relu(tf.nn.xw_plus_b(self.fc_1_output, W3, b3), name="fc-3-out")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")
        
            # ================ Loss and Accuracy ================
        # CalculateMean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(self.scores, self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss
        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
        

In [ ]:
import os
import time
import datetime
def batch_iter(x, y, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    # data = np.array(data)
    data_size = len(x)
    num_batches_per_epoch = int(data_size/batch_size) + 1
    for epoch in range(num_epochs):
        print("In epoch >> " + str(epoch + 1))
        print("num batches per epoch is: " + str(num_batches_per_epoch))
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            x_shuffled = x[shuffle_indices]
            y_shuffled = y[shuffle_indices]
        else:
            x_shuffled = x
            y_shuffled = y
        for batch_num in range(num_batches_per_epoch-1):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            x_batch = x_shuffled[start_index:end_index]
            y_batch = y_shuffled[start_index:end_index]
            batch = list(zip(x_batch, y_batch))
            yield batch

In [ ]:
batch_size =128
num_epochs = 20
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
        allow_soft_placement=True,
        log_device_placement=False)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = CharCNN()
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(
            grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.histogram_summary(
                    "{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.scalar_summary(
                    "{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.merge_summary(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(
            os.path.curdir, "runs_new", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.scalar_summary("loss", cnn.loss)
        acc_summary = tf.scalar_summary("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.merge_summary(
            [loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.train.SummaryWriter(
            train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.merge_summary([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.train.SummaryWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already
        # exists, so we need to create it.
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.all_variables())

        # Initialize all variables
        sess.run(tf.initialize_all_variables())
        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.training:1,
              cnn.dropout_keep_prob: 0.5
            }       
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op,
                    cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            # write fewer training summaries, to keep events file from
            # growing so big.
            if step % (evaluate_every / 2) == 0:
                print("{}: step {}, loss {:g}, acc {:g}".format(
                    time_str, step, loss, accuracy))
                train_summary_writer.add_summary(summaries, step)
        def dev_step(x_batch, y_batch, writer=None):
            dev_size = len(x_batch)
            max_batch_size = 500
            num_batches = dev_size/max_batch_size
            acc = []
            losses = []
            print("Number of batches in dev set is " + str(num_batches))
            for i in range(num_batches):
                x_batch_dev = x_batch[i * max_batch_size:(i + 1) * max_batch_size]
                y_batch_dev = y_batch[i * max_batch_size: (i + 1) * max_batch_size]
                feed_dict = {
                  cnn.input_x: x_batch_dev,
                  cnn.input_y: y_batch_dev,
                  cnn.training:0,
                  cnn.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                acc.append(accuracy)
                losses.append(loss)
                time_str = datetime.datetime.now().isoformat()
                print("batch " + str(i + 1) + " in dev >>" +" {}: loss {:g}, acc {:g}".format(time_str, loss, accuracy))
                if writer:
                    writer.add_summary(summaries, step)
            print("\nMean accuracy=" + str(sum(acc)/len(acc)))
            print("Mean loss=" + str(sum(losses)/len(losses)))
        # just for epoch counting
        num_batches_per_epoch = int(len(x_train)/batch_size) + 1
        # Generate batches
        batches = batch_iter(x_train, y_train,batch_size, num_epochs)
        evaluate_every = 300
        checkpoint_every =300
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("Epoch: {}".format(
                    int(current_step / num_batches_per_epoch)))
                print("")
            if current_step % checkpoint_every == 0:
                path = saver.save(
                    sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))